In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### SILVER LAYER SCRIPT

#### DATA ACCESS USNIG APP

In [0]:
spark.conf.set("fs.azure.account.auth.type.amazingworksdatalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.amazingworksdatalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.amazingworksdatalake.dfs.core.windows.net", "<application-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.amazingworksdatalake.dfs.core.windows.net", "<service_credintial>")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.amazingworksdatalake.dfs.core.windows.net", "https://login.microsoftonline.com/<directory_id>/oauth2/token")

#### DATA LOADING

##### READ DATA CALENDAR

In [0]:
df_cal = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Calendar")

In [0]:
df_sales = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Sales*')

In [0]:
df_cus = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Customers")

In [0]:
df_pd_cat = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Product_Categories")

In [0]:
df_ret = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Returns")

In [0]:
df_terr = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Territories")

In [0]:
df_pd_sub = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/Product_Subcategories")

In [0]:
df_pd = spark.read.format('csv').option("header", True).option("inferSchema", True).load("abfss://bronze@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Product_Categories")

## TRANSFORMATIONS

In [0]:
df_cal = df_cal.withColumn('Month',month(col('Date')))\
            .withColumn('Year',year(col('Date')))

In [0]:
df_cal.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Calendar")\
            .save()

In [0]:
df_cus.withColumn("fullName",concat(col('Prefix'),lit(' '),col('FirstName'),lit(' '),col('LastName'))).display()

In [0]:
df_cus = df_cus.withColumn('fullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('lastName')))

In [0]:
df_cus.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Customers")\
            .save()

In [0]:
df_pd_sub.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_SUbCategories")\
            .save()

In [0]:
df_pd = df_pd.withColumn('ProductSKU',split(col('ProductSKU'),'-')[0])\
                .withColumn('ProductName',split(col('ProductName'),' ')[0])

In [0]:
df_pd.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Products")\
            .save()

In [0]:
df_ret.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Returns")\
            .save()

In [0]:
df_terr.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Territories")\
            .save()

In [0]:
df_sales = df_sales.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_sales = df_sales.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sales = df_sales.withColumn('multiply',col('OrderLineItem')*col('OrderQuantity'))

In [0]:
df_sales.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@amazingworksdatalake.dfs.core.windows.net/AdventureWorks_Sales")\
            .save()

In [0]:
df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order')).display()

In [0]:
df_pd_cat.display()